# GloVe + LSTM model for Sentiment Analisis

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import random
import pickle
import os
import nltk
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf 
import string
import re
import pandas as pd 
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize, pos_tag
from sklearn.model_selection import train_test_split
from google.colab import drive
import nltk
nltk.download('omw-1.4')
drive.mount('/content/drive')
!ls '/content/drive/Shareddrives/Deep Learning/DeepLearning_2022/Final project/Data/'
myDrive = '/content/drive/Shareddrives/Deep Learning/DeepLearning_2022/Final project/Data/'
results_path = '/content/drive/Shareddrives/Deep Learning/DeepLearning_2022/Final project/Results/'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


Mounted at /content/drive
 all_words.txt	 data_clean.csv		 'IMDB Dataset.csv'   sorted_words.txt
 data_1		 data_no_stop_words.csv   inputs_ids.pt


Load dataset preprocessed and dictionaries to convert words to their corresponding id

In [ ]:
data = pd.read_csv(f'{myDrive}data_clean.csv')
file1 = open(f'{myDrive}all_words.txt', 'r')
file2 = open(f'{myDrive}sorted_words.txt', 'r')
all_words = file1.read().splitlines()
count_words = Counter(all_words)
sorted_words=count_words.most_common(len(all_words))
vocab_to_int={w:i+1 for i,(w,c) in enumerate(sorted_words)}


Change the label from string to int to have a binary output

In [ ]:
data.sentiment = [0 if each == "negative" else 1 for each in data.sentiment]

Tranform reviews from text to a list of ints using the dictionary vocab_to_int

In [ ]:
def encode_review(review):
  encoded_review=list()
  words = review.split(' ')
  for word in words:
    encoded_review.append(vocab_to_int[word])

  return encoded_review

In [ ]:
data2 = data.copy()
data['review'] = data.apply(lambda row: encode_review(row['review']), axis = 1)
data['review_length'] = data.apply(lambda row: len(row['review']), axis = 1)

The sequence length of our sentences should be the same. Thus, we use two techniques: 
1. Calculate the average length of reviews (in words) and truncate or pad each review to have such amount of words. 
2. Calculate the maximum length of reviews and add padding to the rest so that all have this length.

In [ ]:
sequence_length = int(sum(list(data['review_length']))/data.shape[0]) # average length
sequence_length_max = max(list(data['review_length']))
print(sequence_length)

def truncate_or_pad(review):
  num_words = len(review)
  if num_words<sequence_length:
    zeros = list(np.zeros(sequence_length-num_words))
    new = zeros+review
  else:
    new = review[:sequence_length]

  return new

def pad(review):
  num_words = len(review)
  new = review
  if num_words<sequence_length_max:
    zeros = list(np.zeros(sequence_length_max-num_words))
    new = zeros+review
  return new

#test = truncate_or_pad(data.review[0])
data_pad = data.copy()
data['review'] = data.apply(lambda row: truncate_or_pad(row['review']), axis = 1)
data_pad['review'] = data_pad.apply(lambda row: pad(row['review']), axis = 1)

118


Store the reviews in a matrix that will be the input data of our model

In [ ]:
data_matrix = np.zeros((data.shape[0], sequence_length), dtype=int)

for i in range(data.shape[0]):
  data_matrix[i, :] = np.array(data.review[i])

print(data_matrix[0:15, :])

[[    3  1020   326 ...    23  1356    12]
 [    0     0     0 ...  1816  1676    19]
 [    0     0     0 ...    13     6   122]
 ...
 [ 3771   105   110 ...   631     2   546]
 [    0     0     0 ...    23    13   341]
 [    0     0     0 ...  1104 11136  6458]]
85


In [ ]:
data_matrix_pad = np.zeros((data_pad.shape[0], sequence_length_max), dtype=int)

for i in range(data_pad.shape[0]):
  data_matrix_pad[i, :] = np.array(data_pad.review[i])
  
print(data_matrix_pad[0:15, :])

[[    0     0     0 ...   469  3431   323]
 [    0     0     0 ...  1816  1676    19]
 [    0     0     0 ...    13     6   122]
 ...
 [    0     0     0 ...   179    90   160]
 [    0     0     0 ...    23    13   341]
 [    0     0     0 ...  1104 11136  6458]]
85


# GloVe embeding

We will use a GloVe, a pretrain word embedding to train the RNN model.

In [ ]:
!wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
!unzip glove.6B.zip

--2022-06-15 19:19:42--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2022-06-15 19:19:42--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2022-06-15 19:19:43--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [applic

In [ ]:
glove = pd.read_csv('glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove_embedding = {key: val.values for key, val in glove.T.items()} # dictionary of a word to embedding array

The following function creates a embedding matrix given a map of words to indices, the GloVe's embeddings and the dimension of the embeddings

In [ ]:
def create_word_embedding(words,glove_dict,dimension):
  embedding_matrix=np.zeros((len(words)+1,dimension)) #
  
  for word,index in words.items():
    if word in glove_dict:
      embedding_matrix[index]=glove_dict[word]
  return embedding_matrix

Then, we want to store the distinct words ordered by occurrencies

In [ ]:
unique_words =  list(map(lambda x: x[0], sorted_words)) # pick first element (word) of each tuple in sorted_words

Let's construct a matrix of embeddings for our vocabulary of words

In [ ]:
tokenizer=tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(unique_words)

text_token=tokenizer.texts_to_sequences(unique_words)
print(tokenizer.word_index)
word_embedding = create_word_embedding(tokenizer.word_index, glove_embedding, dimension=100)

{'movie': 1, 'film': 2, 'one': 3, 'make': 4, 'like': 5, 'see': 6, 'get': 7, 'time': 8, 'would': 9, 'good': 10, 'character': 11, 'watch': 12, 'go': 13, 'story': 14, 'even': 15, 'really': 16, 'think': 17, 'show': 18, 'well': 19, 'scene': 20, 'look': 21, 'much': 22, 'say': 23, 'end': 24, 'know': 25, 'could': 26, 'bad': 27, 'people': 28, 'great': 29, 'also': 30, 'first': 31, 'give': 32, 'take': 33, 'way': 34, 'love': 35, 'play': 36, 'thing': 37, 'come': 38, 'find': 39, 'life': 40, 'seem': 41, 'work': 42, 'plot': 43, 'two': 44, 'many': 45, 'actor': 46, 'year': 47, 'want': 48, 'never': 49, 'best': 50, 'try': 51, 'little': 52, 'ever': 53, 'man': 54, 'better': 55, 'still': 56, 'act': 57, 'part': 58, 'something': 59, 'director': 60, 'back': 61, 'use': 62, 'lot': 63, 'u': 64, 'real': 65, 'guy': 66, 'performance': 67, 'woman': 68, 'old': 69, 'though': 70, 'funny': 71, 'another': 72, 'star': 73, 'actually': 74, 'nothing': 75, 'role': 76, 'tell': 77, 'start': 78, 'leave': 79, 'new': 80, 'feel': 81,

These are the weights we use to initialize our embedding layer, some are still vectors of 0 because some words in the training dataset were not part of GloVe's dictionary 

In [ ]:
print(word_embedding)

[[ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.38251   0.14821   0.60601  ...  0.058921  0.091112  0.47283 ]
 [ 0.19916  -0.049702  0.24579  ... -0.068109  0.017651  0.06455 ]
 ...
 [-0.15927  -0.37157  -0.24017  ... -0.20835  -0.28694   0.27697 ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.027842  0.59833   0.53633  ...  0.12272   0.48018   0.54391 ]]


# Create and train LSTM model 

Now, split into train and test datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_matrix_pad, list(data_pad.sentiment), test_size=0.3)

In [ ]:
train_data=TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
test_data=TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))

Divide the data into batches

In [ ]:
batch_size=50
train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader=DataLoader(test_data, batch_size=batch_size, shuffle=True)

Let's define our LSTM neural network

In [ ]:
 
class SentimentalLSTM(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):    
        """
        Initialize the model by setting up the layers
        """
        super().__init__()
        self.output_size=output_size
        self.n_layers=n_layers
        self.hidden_dim=hidden_dim
        
        #Embedding and LSTM layers
        self.embedding=nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight = nn.Parameter(torch.tensor(word_embedding,dtype=torch.float32))
        self.lstm=nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        
        #dropout layer
        self.dropout=nn.Dropout(0.5)
        
        #Linears and sigmoid layer
        self.fc1=nn.Linear(hidden_dim, 64)
        self.fc2=nn.Linear(64, 16)
        self.fc3=nn.Linear(16,output_size)
        self.sigmoid=nn.Sigmoid()
        
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size=x.size()
        
        #Embadding and LSTM output
        embedd=self.embedding(x)
        lstm_out, hidden=self.lstm(embedd, hidden)
        
        #stack up the lstm output
        lstm_out=lstm_out.contiguous().view(-1, self.hidden_dim)
        
        #dropout and fully connected layers
        out=self.dropout(lstm_out)
        out=self.fc1(out)
        out=self.dropout(out)
        out=self.fc2(out)
        out=self.dropout(out)
        out=self.fc3(out)
        sig_out=self.sigmoid(out)
        
        sig_out=sig_out.view(batch_size, -1)
        sig_out=sig_out[:, -1]
        
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        """Initialize Hidden STATE"""
        # Create two new sensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        return hidden

The following function will be used to train the model using train dataset

In [ ]:
def train_loop(model, model_name = 'model.ckpt', device = 'cuda'):
    model.train()
    total_step = len(train_loader)
    losses_list = []
    criterion = nn.BCELoss()

    for epoch in range(num_epochs):
        loss_avg = 0
        h = net.init_hidden(batch_size)
        nBatches = 0
        # TRAINING LOOP
        for i, (review, sentiment) in enumerate(train_loader):
            review = review.type(torch.LongTensor).to(device)
            sentiment = sentiment.to(device)

            h = tuple([each.data for each in h])
            net.zero_grad()

            # Forward pass
            outputs,h = model(review, h)
            # Calculate CE_loss
            loss = criterion(outputs, sentiment)

            # Backpropagate
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_avg += loss.cpu().item()
            nBatches+=1
            if (i+1) % 200 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss_avg / nBatches))
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss_avg / nBatches))
        losses_list.append(loss_avg / nBatches)
        torch.save(model.state_dict(), results_path+model_name)


Test function to use after train, with the test dataset

In [ ]:
#Load and test function
def load_and_test(model, device='cuda', model_name = 'model.ckpt'):
  model.load_state_dict(torch.load(results_path+model_name))
  criterion = nn.BCELoss()
  h = net.init_hidden(batch_size)
  # Test the model
  model.eval() # Set the model in evaluation mode

  # Compute testing accuracy
  with torch.no_grad():
      correct = 0
      total = 0
      for review, sentiment in test_loader:
          h = tuple([each.data for each in h])

          review = review.type(torch.LongTensor).to(device)
          sentiment = sentiment.to(device)
          # get network predictions
          outputs, h = model(review, h)

          # get predicted class
          predicted = torch.round(outputs)
          # compare with the ground-truth
          total += sentiment.size(0)
          correct += (predicted == sentiment).sum().item()

      print('Test Accuracy of the model: {} %'.format(100 * correct / total))

Instantiation of the model with hyperparams:

In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = word_embedding.shape[0]
output_size = 1
embedding_dim = word_embedding.shape[1]
hidden_dim = 256
n_layers = 2


Checking different values for Adam SGD optimizer

In [ ]:
num_epochs = 6
for lr_i in [0.001, 0.0005, 0.002]:
  net = SentimentalLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
  net = net.cuda()
  optimizer = torch.optim.Adam(net.parameters(),lr = lr_i)

  # Device configuration (choose GPU if it is available )
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

  train_loop(net, model_name = 'model_4.ckpt')
  load_and_test(net,  model_name = 'model_4.ckpt')

Epoch [1/6], Step [200/700], Loss: 0.5771
Epoch [1/6], Step [400/700], Loss: 0.5288
Epoch [1/6], Step [600/700], Loss: 0.4875
Epoch [1/6], Step [700/700], Loss: 0.4830
Epoch [2/6], Step [200/700], Loss: 0.3211
Epoch [2/6], Step [400/700], Loss: 0.3021
Epoch [2/6], Step [600/700], Loss: 0.2974
Epoch [2/6], Step [700/700], Loss: 0.2888
Epoch [3/6], Step [200/700], Loss: 0.1556
Epoch [3/6], Step [400/700], Loss: 0.1580
Epoch [3/6], Step [600/700], Loss: 0.1596
Epoch [3/6], Step [700/700], Loss: 0.1602
Epoch [4/6], Step [200/700], Loss: 0.0803
Epoch [4/6], Step [400/700], Loss: 0.0751
Epoch [4/6], Step [600/700], Loss: 0.0799
Epoch [4/6], Step [700/700], Loss: 0.0804
Epoch [5/6], Step [200/700], Loss: 0.0395
Epoch [5/6], Step [400/700], Loss: 0.0389
Epoch [5/6], Step [600/700], Loss: 0.0396
Epoch [5/6], Step [700/700], Loss: 0.0459
Epoch [6/6], Step [200/700], Loss: 0.0698
Epoch [6/6], Step [400/700], Loss: 0.0560
Epoch [6/6], Step [600/700], Loss: 0.0516
Epoch [6/6], Step [700/700], Loss:

Checking different values for momentum SGD optimizer

In [ ]:
for lr_i in [0.001, 0.01, 0.05, 0.06]:
  net = SentimentalLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
  net = net.cuda()
  optimizer = torch.optim.SGD(net.parameters(), lr=lr_i, momentum=0.9)

  # Device configuration (choose GPU if it is available )
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

  train_loop(net, model_name = 'model_4.ckpt')
  load_and_test(net,  model_name = 'model_4.ckpt')

Epoch [1/6], Step [200/700], Loss: 0.6942
Epoch [1/6], Step [400/700], Loss: 0.6940
Epoch [1/6], Step [600/700], Loss: 0.6939
Epoch [1/6], Step [700/700], Loss: 0.6938
Epoch [2/6], Step [200/700], Loss: 0.6934
Epoch [2/6], Step [400/700], Loss: 0.6932
Epoch [2/6], Step [600/700], Loss: 0.6931
Epoch [2/6], Step [700/700], Loss: 0.6931
Epoch [3/6], Step [200/700], Loss: 0.6935
Epoch [3/6], Step [400/700], Loss: 0.6932
Epoch [3/6], Step [600/700], Loss: 0.6931
Epoch [3/6], Step [700/700], Loss: 0.6930
Epoch [4/6], Step [200/700], Loss: 0.6926
Epoch [4/6], Step [400/700], Loss: 0.6928
Epoch [4/6], Step [600/700], Loss: 0.6928
Epoch [4/6], Step [700/700], Loss: 0.6928
Epoch [5/6], Step [200/700], Loss: 0.6928
Epoch [5/6], Step [400/700], Loss: 0.6928
Epoch [5/6], Step [600/700], Loss: 0.6927
Epoch [5/6], Step [700/700], Loss: 0.6928
Epoch [6/6], Step [200/700], Loss: 0.6923
Epoch [6/6], Step [400/700], Loss: 0.6923
Epoch [6/6], Step [600/700], Loss: 0.6924
Epoch [6/6], Step [700/700], Loss:

In [ ]:
num_epochs = 6
for mom in [0.8, 0.85, 0.89, 0.95]:
  net = SentimentalLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
  net = net.cuda()
  optimizer = torch.optim.SGD(net.parameters(), lr=0.06, momentum=mom)

  # Device configuration (choose GPU if it is available )
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

  train_loop(net, model_name = 'model_4.ckpt')
  load_and_test(net,  model_name = 'model_4.ckpt')

Epoch [1/6], Step [200/700], Loss: 0.6945
Epoch [1/6], Step [400/700], Loss: 0.6934
Epoch [1/6], Step [600/700], Loss: 0.6846
Epoch [1/6], Step [700/700], Loss: 0.6762
Epoch [2/6], Step [200/700], Loss: 0.5848
Epoch [2/6], Step [400/700], Loss: 0.5760
Epoch [2/6], Step [600/700], Loss: 0.5789
Epoch [2/6], Step [700/700], Loss: 0.5734
Epoch [3/6], Step [200/700], Loss: 0.5180
Epoch [3/6], Step [400/700], Loss: 0.4927
Epoch [3/6], Step [600/700], Loss: 0.4737
Epoch [3/6], Step [700/700], Loss: 0.4642
Epoch [4/6], Step [200/700], Loss: 0.3850
Epoch [4/6], Step [400/700], Loss: 0.3758
Epoch [4/6], Step [600/700], Loss: 0.3692
Epoch [4/6], Step [700/700], Loss: 0.3663
Epoch [5/6], Step [200/700], Loss: 0.3398
Epoch [5/6], Step [400/700], Loss: 0.3356
Epoch [5/6], Step [600/700], Loss: 0.3374
Epoch [5/6], Step [700/700], Loss: 0.3340
Epoch [6/6], Step [200/700], Loss: 0.3219
Epoch [6/6], Step [400/700], Loss: 0.3169
Epoch [6/6], Step [600/700], Loss: 0.3103
Epoch [6/6], Step [700/700], Loss: